# Iceberg Lab 
## Unit 3: CRUD Support

This lab is powered by Dataproc Serverless Spark.

In the previous unit, we -
1. Create Iceberg table in Hadoop Catalog
2. Explore folder structure and version contorl in Hadoop Catalog
3. Create Partitioned and Unpartitioned Icberg Table in Hive Catalog
4. Explore folder structure in Hive Catalog
5. Explore table descriptions and metadata files for tables in both catalogs


In this unit, we will learn how to -
1. Delete a record and study the metadata log
2. Insert a record and study the metadata log
3. Update a record and study the metadata log
4. Insert Overwrite from a source table and study the metadata log
5. Merge from a source table and study the metadata log 

### 1. Imports

In [2]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [3]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spark

### 3. Declare variables

In [4]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  nikhim-iceberg-lab


In [5]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  nikhim-iceberg-lab


In [6]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  928505941962


In [7]:
DPMS_NAME=f"iceberg-hms-{PROJECT_NUMBER}"
LOCATION="us-central1"

metastore_dir = !gcloud metastore services describe $DPMS_NAME --location $LOCATION |grep 'hive.metastore.warehouse.dir'| cut -d':' -f2- | xargs 
HIVE_METASTORE_WAREHOUSE_DIR = metastore_dir[0]

print("HIVE_METASTORE_WAREHOUSE_DIR",HIVE_METASTORE_WAREHOUSE_DIR)

HIVE_METASTORE_WAREHOUSE_DIR gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse


In [8]:
TABLE_NAME="loans_by_state_iceberg"
DB_NAME="loan_db"
#fully qualified table name
FQTN=f"{DB_NAME}.{TABLE_NAME}"
print("Fully quailified table name :",FQTN)

loan_db.loans_by_state_iceberg


In [9]:
# Checking the initial folder structure

!gsutil ls -r {HIVE_METASTORE_WAREHOUSE_DIR}/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-b2d1021e-d3ad-4aff-baa7-1c740e8a3144-00001.parquet

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00000-e9f97377-9cf7-4d6b-a97f-cde34a16cb51.metadata.json
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/d37b2f20-215b-4690-9fdf-1e59a8e68f24-m0.avro
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/lo

### 4. DELETE operation

In [10]:
spark.sql(f"SELECT * FROM {FQTN}").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |10318     |
|SC        |5460      |
|LA        |5284      |
|MN        |8031      |
|NJ        |16367     |
|DC        |1059      |
|OR        |5258      |
|VA        |12775     |
|RI        |1968      |
|KY        |4287      |
|WY        |964       |
|NH        |2148      |
|MI        |11638     |
|NV        |6309      |
|WI        |5798      |
|ID        |522       |
|CA        |62090     |
|CT        |6767      |
|NE        |1299      |
|MT        |1220      |
+----------+----------+
only showing top 20 rows



In [12]:
spark.sql(f"DELETE FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

++
||
++
++



In [13]:
spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
+----------+----------+



In [14]:
# Explore the table folder contents after DELETE Operation
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-b2d1021e-d3ad-4aff-baa7-1c740e8a3144-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00054-4-595dbb6c-4517-4463-91be-a1b513cf82f2-00001.parquet

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00000-e9f97377-9cf7-4d6b-a97f-cde34a16cb51.metadata.json
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-wa

**NOTE:** 
1. In Data folder, a new  parquet file is added which has all records of addr_state="AZ" removed
2. In Metadata folder, we have new metadata, manifest lists and manifest files 

In [15]:
#Fetching the latest metadata file

#Note: In production usage the latest version of Metadata file is not always the one with highest sequence number. As we see further in lab Time Travel can restore an earlier version of metadata as current.

latest_metadata = !gsutil ls  $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/metadata/*.metadata.json | tail -1
LATEST_METADATA_FILE = latest_metadata[0]
print("LATEST_METADATA_FILE", LATEST_METADATA_FILE)

LATEST_METADATA_FILE gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00001-3278c0db-1d9d-4483-9b6f-e6fe9bc799c1.metadata.json


In [16]:
#Explore the contents of the latest metadata file

#Note that the new metadata files keeps track of new snapshots and also indicates the no of files deleted or modified 

!gsutil cat {LATEST_METADATA_FILE}

{
  "format-version" : 1,
  "table-uuid" : "d08425df-63be-497b-b873-529790a5c137",
  "location" : "gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg",
  "last-updated-ms" : 1676068185022,
  "last-column-id" : 2,
  "schema" : {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  },
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  } ],
  "partition-spec" : [ ],
  "default-spec-id" : 0,
  "partition-specs" : [ {
    "spec-id" : 0,
    "fields" : [

### 5. INSERT Operation


In [17]:
spark.sql(f"INSERT INTO {FQTN} VALUES ('AZ',50000)")

DataFrame[]

In [18]:
spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |50000     |
+----------+----------+



In [19]:
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-b2d1021e-d3ad-4aff-baa7-1c740e8a3144-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-205-c4327bde-de7b-4bb7-a2fe-cb90b1c77623-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00054-4-595dbb6c-4517-4463-91be-a1b513cf82f2-00001.parquet

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-wa

**NOTE:** INSERT operation has created new set of files in the table directory
1. New parquet file in data folder with new row for "AZ" 
2. New snapshot, manifest, meta-data files in the meta-data folder

In [20]:
#Fetching the latest metadata file

latest_metadata = !gsutil ls  $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/metadata/*.metadata.json | tail -1
LATEST_METADATA_FILE = latest_metadata[0]
print("LATEST_METADATA_FILE", LATEST_METADATA_FILE)

LATEST_METADATA_FILE gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00002-bc68f99b-7d12-4b18-a2b2-e9c9e745cbfe.metadata.json


In [21]:
#Explore contents of latest metadata file post INSERT operation

!gsutil cat {LATEST_METADATA_FILE}

{
  "format-version" : 1,
  "table-uuid" : "d08425df-63be-497b-b873-529790a5c137",
  "location" : "gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg",
  "last-updated-ms" : 1676068386455,
  "last-column-id" : 2,
  "schema" : {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  },
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  } ],
  "partition-spec" : [ ],
  "default-spec-id" : 0,
  "partition-specs" : [ {
    "spec-id" : 0,
    "fields" : [

**NOTE:**
1. The value of current snapshot is set to uuid of the latest snapshot file
2. The operation is indicated as "append" and 
3. Note the number of files added/deleted as part of this INSERT operation. 
    
    
   In our specific case since we are using unpartitioned table here all 51 records are part of the same file hence the entire file is being overwriten. 


### 6. UPDATE Operation

In [23]:
spark.sql(f"UPDATE {FQTN} SET loan_count = 11111 WHERE addr_state='AZ'").show(truncate=False)

++
||
++
++



In [24]:
spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |11111     |
+----------+----------+



In [25]:
# Note how the update created a new parquet file and added more metadata files in the metadata directory
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-b2d1021e-d3ad-4aff-baa7-1c740e8a3144-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-205-c4327bde-de7b-4bb7-a2fe-cb90b1c77623-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00054-4-595dbb6c-4517-4463-91be-a1b513cf82f2-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00130-209-e5d9f868-2616-41fc-9233-464482068f34-00001.parquet

gs://gcs-bucket-icebe

### 7. INSERT OVERWRITE Operation

In [26]:
#Creating a source table for overwriting 

overwrt_df = spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state NOT IN ('IA','CA')")
overwrt_df.createOrReplaceTempView("overwrite_source_table")

In [27]:
#Overwrite Iceberg table with the values from above source table
spark.sql(f"INSERT OVERWRITE {FQTN} SELECT * FROM overwrite_source_table").show(truncate=False)

DataFrame[]

In [28]:
#
spark.sql(f"SELECT COUNT(*) as count  from {FQTN}").show(truncate=False)

+-----+
|count|
+-----+
|49   |
+-----+



In [29]:
#Fetching the latest metadata file

latest_metadata = !gsutil ls  $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/metadata/*.metadata.json | tail -1
LATEST_METADATA_FILE = latest_metadata[0]
print("LATEST_METADATA_FILE", LATEST_METADATA_FILE)

!gsutil cat {LATEST_METADATA_FILE} 

LATEST_METADATA_FILE gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00004-1f15486a-5485-4bdf-aefb-e1b7a9da220f.metadata.json
{
  "format-version" : 1,
  "table-uuid" : "d08425df-63be-497b-b873-529790a5c137",
  "location" : "gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg",
  "last-updated-ms" : 1676068590093,
  "last-column-id" : 2,
  "schema" : {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  },
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,

**NOTE:**
INSERT OVERWRITE operation performs an atomic overwrite on the data file.
Notice the operation summary of the current snapshot in above metadata file

### 8. MERGE INTO Operation

In [36]:
#Create a Source Dataframe to merge
toBeMergedRows = [('IA', 11111), ('CA', 11111), ('IN', 11111)]
toBeMergedColumns = ['addr_state', 'loan_count']
toBeMergedDF = spark.createDataFrame(toBeMergedRows, toBeMergedColumns)
toBeMergedDF.createOrReplaceTempView("to_be_merged_table")
toBeMergedDF.orderBy("addr_state").show(3)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|        CA|     11111|
|        IA|     11111|
|        IN|     11111|
+----------+----------+



In [33]:
#Iceberg Table before merging
spark.sql(f"SELECT addr_state,loan_count FROM {FQTN} WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|IN        |7511      |
+----------+----------+



In [34]:
#build merge statement
mergeSQLStatement = f"MERGE INTO {FQTN} as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * "

print(mergeSQLStatement)

MERGE INTO loan_db.loans_by_state_iceberg as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * 


In [37]:
spark.sql(mergeSQLStatement).show(truncate=False)

++
||
++
++



In [39]:
# Iceberg Table 
spark.sql(f"SELECT addr_state,loan_count FROM {FQTN} WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|CA        |11111     |
|IA        |11111     |
|IN        |11111     |
+----------+----------+



In [40]:
# Merge created new set of files 
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-b2d1021e-d3ad-4aff-baa7-1c740e8a3144-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-205-c4327bde-de7b-4bb7-a2fe-cb90b1c77623-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-410-3265a2ac-0164-4598-a86e-03a63e191071-00001.parquet
gs://gcs-bucket-iceberg-hms-928505941962-71d67f3e-cf27-4b25-a996-86a/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-457-db97e1c8-4cc1-4098-94d7-a1172491a65c-00001.parquet
gs://gcs-bucket-iceb

23/02/10 22:46:36 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /var/tmp/spark/local-dir/blockmgr-77e65e46-a4ea-4ec4-b7cf-132abf558eb6. Falling back to Java IO way
java.io.IOException: Failed to delete: /var/tmp/spark/local-dir/blockmgr-77e65e46-a4ea-4ec4-b7cf-132abf558eb6
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1193)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:318)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:314)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.c

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK